# **Deep Research with Bing Search**

This notebook demonstrates an agentic research workflow that leverages Azure AI services to conduct comprehensive web-based research on any topic. The workflow includes:

1. **Research Planning** - Breaking down complex queries into structured subtopics and targeted search queries
2. **Information Retrieval** - Using Bing Search API through Azure AI Services to gather relevant web content
3. **Content Analysis** - Summarizing search results and extracting key insights 
4. **Report Generation** - Creating detailed research reports with proper citations
5. **Peer Review** - Evaluating report quality and suggesting improvements until quality standards are met

The notebook orchestrates multiple specialized AI agents working together:
- PlannerAgent - Creates comprehensive research plans with subtopics and queries
- BingSearchAgent - Retrieves relevant search results from the web
- SummaryAgent - Extracts key insights from retrieved content
- ResearchAgent - Compiles findings into structured research reports
- PeerReviewAgent - Provides quality feedback in a continuous improvement loop

Built with Azure OpenAI, Azure AI Agent Service

## Environment Setup

First, we'll set up our environment by importing necessary libraries and loading environment variables from a .env file. These environment variables contain configuration details such as API keys and endpoints for the Azure OpenAI and Bing Search services.

In [ ]:
import dotenv
dotenv.load_dotenv(".env", override=True)

True

### Azure AI Foundry Connections

First, we'll establish connections to Azure AI Projects, which provides the infrastructure for our Bing Search agent.

In [1]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
import os

project_client = AIProjectClient(
    credential=DefaultAzureCredential(),
    endpoint=os.getenv("PROJECT_ENDPOINT")
)

The following cell will **create the Azure AI Agents**, so you only need to run this cell **once**.

In [2]:
# from common.create_azure_ai_agents import (
#     create_bing_search_agent,
#     create_research_plan_agent,
#     create_summary_agent,
#     create_research_report_agent,
#     create_peer_review_agent
# )

# planner_agent = create_research_plan_agent(project_client=project_client)
# bing_search_agent = create_bing_search_agent(project_client=project_client)
# summary_agent = create_summary_agent(project_client=project_client)
# research_agent = create_research_report_agent(project_client=project_client)
# peer_review_agent = create_peer_review_agent(project_client=project_client)

Fetch agents from your AI Foundry Project

In [3]:
planner_agent = project_client.agents.get_agent(agent_id=os.getenv("PlannerAgentID"))
bing_search_agent = project_client.agents.get_agent(agent_id=os.getenv("BingSearchAgentID"))
summary_agent = project_client.agents.get_agent(agent_id=os.getenv("SummaryAgentID"))
research_agent = project_client.agents.get_agent(agent_id=os.getenv("ResearchAgentID"))
peer_review_agent = project_client.agents.get_agent(agent_id=os.getenv("PeerReviewAgentID"))

Update their system messages

In [ ]:
from common.update_instructions import (
    update_planner_instructions,
    update_bing_instructions,
    update_summary_instructions,
    update_research_instructions,
    update_peer_review_instructions
)

planner_agent = update_planner_instructions(agent=planner_agent)
bing_search_agent = update_bing_instructions(agent=bing_search_agent)
summary_agent = update_summary_instructions(agent=summary_agent)
research_agent = update_research_instructions(agent=research_agent)
peer_review_agent = update_peer_review_instructions(agent=peer_review_agent)

## Research Workflow

Our system uses specialized AI agents to transform a user query into a comprehensive research report through these steps:

### Process Flow

1. **User Query** → User submits research topic or question
2. **Planning** → PlannerAgent develops structured research plan with objectives and subtopics
3. **Information Retrieval** → BingSearchAgent executes targeted web searches for each area
4. **Analysis** → SummaryAgent processes results, extracting key insights while preserving technical details
5. **Synthesis** → ResearchAgent creates well-structured report with proper citations
6. **Quality Control** → PeerReviewAgent evaluates report for completeness, clarity, and evidence
7. **Revision** → If needed, research report undergoes improvement cycles based on feedback
8. **Delivery** → Final comprehensive, high-quality report delivered to user

This collaborative approach combines the strengths of different specialized agents to produce thorough, evidence-based research that meets predefined quality standards.

In [ ]:
from common.helper import create_research_workflow_diagram

# This will generate research_workflow_diagram.png and return the Digraph object
workflow_diagram = create_research_workflow_diagram()
workflow_diagram

Let's start with a sample research query.

In [ ]:
# user_query="What big industries will AI have the most affected on?"
user_query="What are the differences between classical machine learning, deep learning and generative AI?"

### Step 1: Research Planning

The PlannerAgent analyzes the research query and creates a structured plan with:

- Research objective - A clear statement of what the research aims to accomplish
- Subtopics - Key areas to explore for comprehensive coverage
- Search queries - Specific queries for each subtopic to gather relevant information
- Success criteria - Metrics to determine research completeness
- Related topics - Additional areas that may provide valuable context

In [ ]:
from azure.ai.agents.models import MessageRole
from common.data_models import ResearchPlan
from common.utils_ai_agents import (
    add_user_message_to_thread,
    invoke_agent
)
import json

# create a thread and add the user message
thread = project_client.agents.threads.create()
add_user_message_to_thread(project_client, thread.id, user_query)

# invoke the planner agent to create a research plan
planner_agent_output, thread = invoke_agent(
    project_client=project_client,
    thread=thread,
    agent=planner_agent
)

# parse the output to a ResearchPlan object
plan_data = json.loads(planner_agent_output)
plan = ResearchPlan(**plan_data)

# delete the thread
project_client.agents.threads.delete(thread_id=thread.id)

In [ ]:
plan.research_tasks[0].search_queries

### Step 2: Information Retrieval

The BingSearchAgent executes web searches for each query in our research plan. For each subtopic:

1. We send multiple search queries to gather diverse perspectives
2. The agent returns structured search results with titles, full_text, and URLs
3. Results are organized by subtopic for further processing

This step leverages Azure AI Projects with Bing Search integration to ensure up-to-date information from across the web.

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from collections import defaultdict
from typing import Dict, Any, List, Tuple
from tqdm import tqdm
from common.utils_search import extract_agent_response_and_urls

MAX_WORKERS = 8  # adjust for your rate limits

def run_one_query(subtopic_name: str, query: str) -> Dict[str, Any]:
    prompt = f"""
    Research the following query: {query}
    This is related to subtopic: {subtopic_name}
    Please provide the information and cite your sources using the available tools.
    """
    thread = None
    try:
        thread = project_client.agents.threads.create()
        add_user_message_to_thread(project_client, thread.id, prompt)

        _out, _ = invoke_agent(
            project_client=project_client,
            thread=thread,
            agent=bing_search_agent
        )

        text, urls = extract_agent_response_and_urls(project_client, thread.id, query)
        return {"query": query, "agent_response": text, "results": urls}
    except Exception as e:
        return {"query": query, "results": [], "error": str(e)}
    finally:
        try:
            if thread is not None:
                project_client.agents.threads.delete(thread_id=thread.id)
        except Exception:
            pass

# Flatten tasks
# si: index of the subtopic
# qi: index of the query within that subtopic
# st.subtopic: the subtopic name
# q: the query text

tasks: List[Tuple[int, int, str, str]] = [
    (si, qi, st.subtopic, q)
    for si, st in enumerate(plan.research_tasks)
    for qi, q in enumerate(st.search_queries)
]

# Run in parallel
results = defaultdict(dict)  # results[si][qi] = entry
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
    fmap = {ex.submit(run_one_query, subtopic_name, query): (si, qi)
            for si, qi, subtopic_name, query in tasks}
    for fut in tqdm(as_completed(fmap), total=len(fmap), desc="Running research queries in parallel"):
        si, qi = fmap[fut]
        try:
            results[si][qi] = fut.result()
        except Exception as e:
            results[si][qi] = {"query": tasks[si][3], "results": [], "error": str(e)}

# Rebuild in original shape and order
search_results: List[Dict[str, Any]] = []
for si, st in enumerate(plan.research_tasks):
    queries = [results[si].get(qi, {"query": q, "results": [], "error": "Missing result"})
               for qi, q in enumerate(st.search_queries)]
    search_results.append({"subtopic": st.subtopic, "queries": queries})

# Quick status
for block in search_results:
    ok = sum(1 for q in block["queries"] if "error" not in q)
    print(f"{block['subtopic']}: {ok}/{len(block['queries'])} queries succeeded")


In [ ]:
print(f"Planned total search queries: {sum(1 for task in plan.research_tasks for search_query in task.search_queries)}\n")
print(f"Actually total search queries: {sum(1 for task in search_results for result in task['queries'])}\n")

### Step 3: Content Analysis and Summarization

For each search result retrieved, the SummaryAgent:

1. Extracts key facts, statistics, and insights from the raw search content
2. Preserves important technical details, dates, and domain-specific terminology
3. Formats the summary with key insights and detailed paragraph explanations
4. Tracks citations for proper attribution in the final report

This step transforms raw search data into structured, information-rich summaries that will form the basis of our research report.

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import Dict, Any, List, Tuple
from tqdm import tqdm
from common.utils_summary import collect_responses_and_citations

MAX_WORKERS_SUMMARY = 5

def summarize_one(subtopic_result: Dict[str, Any]) -> Dict[str, Any]:
    all_responses, unique_citations = collect_responses_and_citations(subtopic_result)
    content = "\n\n---\n\n".join(all_responses)

    summary = "No content found to summarize for this subtopic."
    thread = None
    if content:
        summary_prompt = (
            f"Summarize the following information related to the subtopic "
            f"'{subtopic_result.get('subtopic', 'Unknown Subtopic')}':\n\n{content}"
        )
        try:
            thread = project_client.agents.threads.create()
            add_user_message_to_thread(project_client, thread.id, summary_prompt)
            out, _ = invoke_agent(project_client=project_client, thread=thread, agent=summary_agent)
            summary = out.strip()
        except Exception as e:
            sub = subtopic_result.get('subtopic', 'Unknown Subtopic')
            summary = f"Error during summarization for subtopic '{sub}'. Details: {e}"
        finally:
            try:
                if thread is not None:
                    project_client.agents.threads.delete(thread_id=thread.id)
            except Exception:
                pass

    citations_list = [{"title": t, "url": u} for (t, u) in unique_citations]
    return {
        "subtopic": subtopic_result.get("subtopic", "Unknown Subtopic"),
        "summary": summary,
        "citations": citations_list,
    }

# Run all subtopics in parallel and preserve order
mapped_chunks: List[Dict[str, Any]] = [None] * len(search_results)

with ThreadPoolExecutor(max_workers=MAX_WORKERS_SUMMARY) as ex:
    fmap = {ex.submit(summarize_one, subtopic_result): i
            for i, subtopic_result in enumerate(search_results)}
    for fut in tqdm(as_completed(fmap), total=len(fmap), desc="Summarizing subtopics in parallel"):
        i = fmap[fut]
        try:
            mapped_chunks[i] = fut.result()
        except Exception as e:
            sub = search_results[i].get("subtopic", "Unknown Subtopic")
            mapped_chunks[i] = {
                "subtopic": sub,
                "summary": f"Error during summarization for subtopic '{sub}'. Details: {e}",
                "citations": [],
            }

# Optional: quick status
ok = sum(1 for m in mapped_chunks if m and not m["summary"].startswith("Error during summarization"))
print(f"Summaries completed: {ok}/{len(mapped_chunks)}")


### Step 4: Report Generation and Peer Review

In this final stage:

1. The ResearchAgent synthesizes all summarized content into a comprehensive report
2. The PeerReviewAgent evaluates the report based on completeness, clarity, evidence, and insight
3. If needed, the report is revised based on feedback
4. This cycle continues until quality standards are met

The final report is structured as a cohesive academic-style document with proper citations and a references section.

In [ ]:
def print_thread_messages(thread):
    messages = project_client.agents.messages.list(thread_id=thread.id)
    for m in messages:
        print(f"roll: {m.role}")
        print(f"agent_id: {m.agent_id}")
        print(f"content: {m.content[0]['text']['value']}")
        print("---")

In [ ]:
from common.data_models import ComprehensiveResearchReport, PeerReviewFeedback
from common.utils_ai_agents import add_user_message_to_thread

def loop_agents(project_client, agent_a, agent_b, initial_input, max_iterations=10):
    """
    Loop between two agents until agent B produces the target output.
    
    Args:
        agent_a: Function that takes input and returns output
        agent_b: Function that takes input and returns output
        initial_input: Starting input for agent A
        max_iterations: Safety limit to prevent infinite loops
    
    Returns:
        The final output from agent B, or None if max iterations reached
    """
    current_input = initial_input
    thread = project_client.agents.threads.create()
    add_user_message_to_thread(project_client, thread.id, current_input)

    for i in range(max_iterations):
        # Agent A processes the input and produces output
        a_output, thread = invoke_agent(
            project_client=project_client,
            thread=thread,
            agent=agent_a
        )

        handover_message = f"A research agent has produced a research report. Please review it."
        add_user_message_to_thread(project_client, thread.id, handover_message)
        
        # Agent B reviews the output
        b_output, thread = invoke_agent(
            project_client=project_client,
            thread=thread,
            agent=agent_b
        )

        b_output_json = json.loads(b_output)
        review = PeerReviewFeedback(**b_output_json)

        # Check if B produced the target output
        if review.is_satisfactory is not False:
            print(f"Target output reached after {i+1} iterations!")
            report_json = json.loads(a_output)
            final_report = ComprehensiveResearchReport(**report_json)

            # delete the thread
            # print_thread_messages(thread)
            project_client.agents.threads.delete(thread_id=thread.id)
            return final_report
        
        # Use B's output as input for the next iteration
        current_input = b_output
        
        handover_message = f"Peer review agent has provided feedback. Please revise the research report based on the feedback."
        add_user_message_to_thread(project_client, thread.id, handover_message)

    # delete the thread
    # print_thread_messages(thread)
    project_client.agents.threads.delete(thread_id=thread.id)
    print(f"Max iterations ({max_iterations}) reached without finding target output")

    return None

In [ ]:
import json
from common.utils_research import preprocess_research_data

research_input = preprocess_research_data(plan, mapped_chunks)
research_input_prompt = json.dumps(research_input, indent=2)

research_query = (
    "Create an exceptionally comprehensive, **paragraph-focused** and detailed research report "
    "using the following content. **Minimize bullet points** and ensure the final text resembles "
    "a cohesive, academic-style paper:\n\n"
    f"{research_input_prompt}\n\n"
    "As a final reminder, don't forget to include the citation list at the end of the report."
)

# Run the loop
final_report = loop_agents(
    project_client=project_client,
    agent_a=research_agent,
    agent_b=peer_review_agent,
    initial_input=research_query,
    max_iterations=10
)

In [ ]:
from IPython.display import display, Markdown
display(Markdown(final_report.research_report))